# 

# Notebook Summary 


### Quickstart

  1. Import etiq library - for install please check our docs (https://docs.etiq.ai/) 

  2. Login to the dashboard - this way you can send the results to your dashboard instance (Etiq AWS instance if you use the SaaS version). To deploy on your own cloud instance, get in touch (info@etiq.ai)

  3. Create or open a project 
  
### Example dataset & model


  4. Example model: using Adult dataset, predicts income above/below 50K 
  
### Create a snapshot 

  5. Log dataset & model 
  
  6. Load a config file
  
  6. Create a snapshot 
  
  
### Scan your snapshot 
  
  7. Run a scan 
  
  8. Retrieve results
  
  
### Example other scans 

# 

# Quickstart 

In [1]:
import etiq


Thanks for trying out the ETIQ.ai toolkit!

Visit our getting started documentation at https://docs.etiq.ai/

Visit our Slack channel at https://etiqcore.slack.com/ for support or feedback.



In [2]:
from etiq import login as etiq_login
etiq_login("https://dashboard.etiq.ai/", "<token>")


(Dashboard supplied updated license information)


Connection successful. Projects and pipelines will be displayed in the dashboard. 😀

In [3]:
# Can get/create a single named project
project = etiq.projects.open(name="Demo Project")

# 

# Example dataset and model

To illustrate some of the library's features, we build a model that predicts whether an applicant makes over or under 50K using the Adult dataset from https://archive.ics.uci.edu/ml/datasets/adult.

First, we'll be encoding the categorical features found in this dataset. 

Second, we'll log the dataset to Etiq. 

In this case we encode prior to splitting into test/train/validate because we know in advance the categories people fall into for this dataset. This means that in production we won't run into new categories that will fall into a bucket not included in this dataset, This allows us to encode prior to splitting into train/test/validation. 

However if this is not the case for your use case, you should NOT encode prior to splitting your sample, as this might lead to LEAKAGE. 

Encoding categorical values itself is problematic as it assigns a numerical ranking to categorical variables. For best practice encoding use one hot encoding. As we limit the free library functionality to 15 features, we will not do one-hot encoding for the purposes of this example. 

Remember: This is an example only. The use case for the majority of scans in Etiq is that you log the model to Etiq once you have the sample that you'll be training on. Usually this sample will have numeric features only as otherwise you will not be able to use it in with the majority of supported libraries training methods. 

In [4]:
# Loading a dataset. We're using the adult dataset
data = etiq.utils.load_sample("adultdata")
data.head()


,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [5]:
from etiq.transforms import LabelEncoder
from collections import Counter
import pandas as pd
import numpy as np 

# use a LabelEncoder to transform categorical variables
cont_vars = ['age', 'educational-num', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']
cat_vars = list(set(data.columns.values) - set(cont_vars))

label_encoders = {}
data_encoded = pd.DataFrame()
for i in cat_vars:
    label = LabelEncoder()
    data_encoded[i] = label.fit_transform(data[i])
    label_encoders[i] = label

data_encoded.set_index(data.index, inplace=True)
data_encoded = pd.concat([data.loc[:, cont_vars], data_encoded], axis=1).copy()



# 

# Create a snapshot

## Log dataset & model to Etiq 

For example of how to use Etiq on already-built models check out our other notebooks. 

In this example we will use a pre-loaded Etiq xgboost wrapper to get you started faster. 
But in your usual use case you will probably scan a model you already built. 

If you are planning to use this in production, not just pre-production get in touch with us. We will release integration demos shortly.


## Loading the config file 

The config is where you can set-up the scans you want, and the thresholds outside of which Etiq finds a problem. In the config you input relevant parameters (e.g. for bias/fairness scans you'll have to tell Etiq which feature is a demographic and which value represents a protected group). 
For more details on the config just check the documentation. You can upload these config files from wherever you want. We provide examples in the Demo repo with each notebook.   

The config gets stored in the database so you can have a log and version control. You will need to load the config before you log the dataset and the model as Etiq will check if it has enough information to process them.



In [ ]:
from etiq.model import DefaultXGBoostClassifier

with etiq.etiq_config("./config_bias.json"):
    
    #load your dataset
    dataset = etiq.BiasDatasetBuilder.dataset(data_encoded)
    
    #load our model
    model = DefaultXGBoostClassifier()
    
    #create a snapshot
    snapshot = project.snapshots.create(name="Test Snapshot", 
                                        dataset=dataset, 
                                        model=model, 
                                        bias_params=etiq.BiasDatasetBuilder.bias_params()
                                       )
    #scan_bias_metrics
    (segments, issues, issue_summary) = snapshot.scan_bias_metrics()

# 

## Bias Metrics & Bias Sources


Scan results are fairly self-explanatory. To help with root cause analysis we give information on:

1) each issue tested as part of the scan 

2) if the issue was found

3) what is the metric the test was based on and what is the value of the metric 

4) what are the thresholds outside which an issue is detected


In [10]:
issues

,name,feature,segment,measure,measure_value,metric,metric_value,threshold
0,demographic_parity_below_threshold,None,all,None,NaN,<function demographic_parity at 0x7fb88fa4da60>,0.186333,"[0.2, 10]"
1,equal_odds_tpr_below_threshold,None,all,None,NaN,<function equal_odds_tpr at 0x7fb88fa4daf0>,0.102702,"(0.5, 1.0)"
2,equal_odds_tnr_below_threshold,None,all,None,NaN,<function equal_odds_tnr at 0x7fb88fa4db80>,0.064248,"[0.2, 10]"
3,equal_opportunity_below_threshold,None,all,None,NaN,<function equal_opportunity at 0x7fb88fa4dc10>,0.102702,"[0.2, 10]"
4,individual_fairness_below_threshold,None,all,None,NaN,<function individual_fairness at 0x7fb88fa4de50>,0.060000,"(0.5, 1.0)"



For more in-depth analysis, you can run our <"issue_type">_sources scan, which will also populate the following: 

5) feature in relation to which a certain issue type was identified (e.g. drift)

6) segment for which a certain issue was detected 

7) feature and segment combination for which an issue was detected

At the moment you also have the option to derive business rules for segments where the issue was identified. This will be covered in notebooks related to the bias scan.


Stay tuned for future releases which will incorporate even more details to help you trouble shoot the issue even faster.




In [11]:
#scan_bias_sources

# Run our pipelines
(segments_sources, issues_sources, issue_summary_sources) = snapshot.scan_bias_sources()

INFO:etiq.pipeline.DataPipeline0945:Starting pipeline
INFO:etiq.pipeline.DataPipeline0945:Computed metrics for the initial dataset
INFO:etiq.pipeline.DataPipeline0945:Completed pipeline
INFO:etiq.pipeline.DebiasPipeline0009:Starting pipeline
INFO:etiq.pipeline.DebiasPipeline0009:Start Phase IdentifyPipeline0091
INFO:etiq.pipeline.IdentifyPipeline0091:Using parent model
INFO:etiq.pipeline.IdentifyPipeline0091:Starting pipeline


/home/raluca/etiq/env/lib/python3.8/site-packages/scipy-1.7.2-py3.8-linux-x86_64.egg/scipy/stats/_entropy.py:72: RuntimeWarning: invalid value encountered in true_divide
  pk = 1.0*pk / np.sum(pk, axis=axis, keepdims=True)
/home/raluca/etiq/env/lib/python3.8/site-packages/numpy-1.21.4-py3.8-linux-x86_64.egg/numpy/lib/function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/raluca/etiq/env/lib/python3.8/site-packages/numpy-1.21.4-py3.8-linux-x86_64.egg/numpy/lib/function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


INFO:etiq.pipeline.IdentifyPipeline0091:Completed pipeline
INFO:etiq.pipeline.DebiasPipeline0009:Completed Phase IdentifyPipeline0091
INFO:etiq.pipeline.DebiasPipeline0009:Computed metrics for the initial dataset
INFO:etiq.pipeline.DebiasPipeline0009:Completed pipeline


In [12]:
issue_summary_sources

,name,metric,measure,features,segments,total_issues_tested,issues_found,threshold
0,missing_sample,None,None,{},"{10, 7}",20,2,"(0.0, 0.0)"
1,low_unpriv_sample,None,None,{},"{0, 1, 2, 3, 4, 5, 6, 8, 9, 11, 12, 13, 14, 15...",18,18,"(0.0, 0.8)"
2,low_priv_sample,None,None,{},{},18,0,"(0.0, 0.8)"
3,skewed_priv_sample,None,None,{},{},13,0,"(0.0, 0.2)"
4,skewed_unpriv_sample,None,None,{},"{0, 1, 5, 6, 9}",18,5,"(0.0, 0.2)"
5,proxy_issue,None,<function corrcoef at 0x7fb8e4085f70>,"{educational-num, race, relationship}","{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",260,22,"(0.0, 0.5)"
6,correlation_issue,None,<function corrcoef at 0x7fb8e4085f70>,"{race, capital-gain, education, hours-per-week...","{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",260,62,"(0.0, 0.2)"
7,low_volume_group,None,None,{},"{4, 7, 9, 10, 12, 14, 19}",20,7,"(1000, inf)"
8,limited_features_issue,<function equal_opportunity at 0x7fb88fa4dc10>,None,{},"{16.0, 19.0}",20,2,"(0.0, 0.2)"


# 

# Example Scans: Accuracy metrics scan

In [ ]:
from etiq.model import DefaultXGBoostClassifier

with etiq.etiq_config("./config_accuracy.json"):
    
    #load your dataset
    dataset = etiq.BiasDatasetBuilder.dataset(data_encoded)
    
    #load our model
    model = DefaultXGBoostClassifier()
    
    #create a snapshot
    snapshot = project.snapshots.create(name="Test Snapshot", 
                                        dataset=dataset, 
                                        model=model, 
                                        bias_params=etiq.BiasDatasetBuilder.bias_params()
                                       )
    #scan_accuracy_metrics
    (segments, issues, issue_summary) = snapshot.scan_accuracy_metrics()

In [16]:
issue_summary

,name,metric,measure,features,segments,total_issues_tested,issues_found,threshold
0,accuracy_below_threshold,<function accuracy at 0x7fb88fa4d8b0>,None,{},{},1,0,"[0.8, 1.0]"
1,accuracy_above_threshold,<function accuracy at 0x7fb88fa4d8b0>,None,{},{},1,0,"[0.8, 1.0]"
2,true_pos_rate_below_threshold,<function true_pos_rate at 0x7fb88fa4d940>,None,{},{},1,0,"[0.6, 1.0]"
3,true_pos_rate_above_threshold,<function true_pos_rate at 0x7fb88fa4d940>,None,{},{},1,0,"[0.6, 1.0]"
4,true_neg_rate_below_threshold,<function true_neg_rate at 0x7fb88fa4d9d0>,None,{},{},1,0,"[0.7, 1.0]"
5,true_neg_rate_above_threshold,<function true_neg_rate at 0x7fb88fa4d9d0>,None,{},{},1,0,"[0.7, 1.0]"


# 